In [1]:
import csv
import time
import urllib
import urllib.request
import numpy as np

from bs4 import BeautifulSoup

url = 'https://karadarefre.jp/Search2/_find/genre:19/page:0'  # URL
num_pages = 1  # page number
request_interval = 5 # interval

data = []
for i in range(0, num_pages + 1):
    url = urllib.parse.urljoin(url, 'page:0'+str(i))    
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    for item_elems in soup.select('div.colum-right-sub-box.r_rich'):
        #URL
        a_elem = item_elems.select('p.clearfix a.link-blue.shop-name')[0]
        web_url = urllib.parse.urljoin(url, a_elem.get('href'))
        data.append(str(web_url))

In [44]:
# 自分の環境では、lxml が使えなかったので、以下の 'html.parser' を使いました。
# soup = BeautifulSoup(html, 'html.parser')
import lxml.html
import requests


data1=[]
for i in range(len(data)):
    url=data[i]
    print('getting page... ', url)
    
    request_interval = 5 # interval
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    for item_elems in soup.select('div.contents'):
        # Shop name
        name_with_html = item_elems.select('a.shop-name')
        for name in name_with_html:
            name=name.getText()
            
        # review score
        score_with_html = item_elems.select('span.review-point')
        for score in score_with_html:
            score=score.getText()
        
        #Station
        station_with_html = item_elems.select('a.link-color-blue.facility-place-name')
        for station in station_with_html:
            station=station.getText()
        
        #Image
        def scrape(image):
            r = requests.get(image)
            raw_html = r.content
            soup = BeautifulSoup(raw_html, 'html.parser')
            links = soup.select('div p > img')
            for link in links:
                image = link.get('scr')

        #brief
        brief_with_html = item_elems.select('p.customer-side-text')
        for brief in brief_with_html:
            brief=brief.getText()
        
        #feature
        feature_with_html = item_elems.select('ul.trouble-type')
        for feature in feature_with_html:
            feature=feature.getText()
        
        
        data1.append({
            'Name': name, 
            'Score': score, 
            "Station" : station,
            'URL': url,
            'Image': image,
            'Brief': brief,
            "Feature": feature
        })
with open('output.csv', 'w', encoding='utf-8') as f:
    # order
    fields = ['Name', 'Score', "Station", 'URL', 'Image', 'Brief', 'Feature']

    writer = csv.DictWriter(f, fieldnames=fields, quoting=csv.QUOTE_NONNUMERIC)
    writer.writeheader()  # header
    writer.writerows(data1)  # data
print('complete')

getting page...  https://karadarefre.jp/facility/175427/?hold=3r
getting page...  https://karadarefre.jp/facility/173578/?hold=3r
getting page...  https://karadarefre.jp/facility/173927/?hold=3r
getting page...  https://karadarefre.jp/facility/176627/?hold=3r
getting page...  https://karadarefre.jp/facility/173360/?hold=3r
getting page...  https://karadarefre.jp/facility/175600/?hold=3r
getting page...  https://karadarefre.jp/facility/173631/?hold=3r
getting page...  https://karadarefre.jp/facility/177020/?hold=3r
getting page...  https://karadarefre.jp/facility/140529/?hold=3r
getting page...  https://karadarefre.jp/facility/177529/?hold=3r
getting page...  https://karadarefre.jp/facility/176834/?hold=3r
getting page...  https://karadarefre.jp/facility/176053/?hold=3r
getting page...  https://karadarefre.jp/facility/174439/?hold=3r
getting page...  https://karadarefre.jp/facility/176189/?hold=3r
getting page...  https://karadarefre.jp/facility/177266/?hold=3r
getting page...  https://

In [45]:
import pandas as pd
pd.read_csv("output.csv")

,Name,Score,Station,URL,Image,Brief,Feature
0,ＢＯＤＹ ＰＥＡＣＥ虎ノ門,4.33,\r\n 虎ノ門駅（港...,https://karadarefre.jp/facility/175427/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n 全身の揉みほぐしをしてもらいました\r\...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\n筋肉痛・関節痛の悩...
1,はりきゅう整体アスイク,4.36,\r\n 塚口駅（阪急...,https://karadarefre.jp/facility/173578/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n 場所はわかりやすかったです。スタッフの方...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\n筋肉痛・関節痛の悩...
2,ｋｉｋｉ整骨院,4.22,\r\n 東大前駅（文...,https://karadarefre.jp/facility/173927/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n 駅近で便利です。サイトの案内通りで着きま...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\nつき指・捻挫・肉離...
3,ｋｉｋｉ整骨院,4.22,\r\n 東大前駅（文...,https://karadarefre.jp/facility/173927/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n 駅近で便利です。サイトの案内通りで着きま...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\nつき指・捻挫・肉離...
4,Ｔｏｔａｌ ｂｏｄｙ ｃａｒｅ Ｓｋｙ 宝塚,4.40,\r\n 仁川駅（西宮...,https://karadarefre.jp/facility/176627/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n 丁寧な説明&施術でとてもリフレッシュでき...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\n筋肉痛・関節痛の悩...
5,センケイ整体院 ダイゴ営業所,4.17,\r\n 石田駅（京都...,https://karadarefre.jp/facility/173360/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n マッサージ、フットマッサージを受けました...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\n筋肉痛・関節痛の悩...
6,第二江坂もりわき骨盤鍼灸整骨院,3.79,\r\n 江坂駅（吹田...,https://karadarefre.jp/facility/175600/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n 駅から２分くらいのはずが、駅からなかなか...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\n筋肉痛・関節痛の悩...
7,AMカイロ＆エレガンスビューティー,3.78,\r\n 渋谷駅（渋谷...,https://karadarefre.jp/facility/173631/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n マンションの1室で、とても狭く女性には厳...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\n筋肉痛・関節痛の悩...
8,自然療法気功整体院「えがおの門」,4.00,\r\n 北大路駅（京...,https://karadarefre.jp/facility/177020/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n 場所が入り組んでて行きづらかった。接客の...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\n筋肉痛・関節痛の悩...
9,野上整体院,3.99,\r\n 室見駅（福岡...,https://karadarefre.jp/facility/140529/?hold=3r,"<p class=""mv-01"">\r\n<img alt="""" class=""lazy"" ...",\r\r\n 初めての気での施術で半信半疑でしたが、こ...,\r\n肩こり・四十肩・五十肩\r\n腰痛・ぎっくり腰\r\n膝痛\r\n筋肉痛・関節痛の悩...
